In [157]:
import pandas as pd
import numpy as np

np.random.seed(42)
df = pd.DataFrame({
    'User_ID': np.arange(1, 1001).reshape(1000),
    'Name': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', None, 'Eve'], size=1000),
    'Age': np.random.choice([25, 30, 35, 40, 45, np.nan, -1], size=1000),
    'Email': np.random.choice(['user@example.com', 'test@domain.com', '', np.nan, 'invalidemail'], size=1000),
    'Signup Date': np.random.choice(['2020-01-01', '2021-05-20', 'not_a_date', np.nan, '2022-03-15'], size=1000),
    'Country': np.random.choice(['USA', 'us', 'Canada', 'CANADA', 'UK', 'N/A', np.nan], size=1000),
    'Income': np.random.normal(50000, 15000, size=1000),
    'Notes': np.random.choice(['Good', 'Bad', 'Average', np.nan, '', ' '], size=1000),
    'Subscribed': np.random.choice(['yes', 'no', 'YES', 'No', 'n', 'Y', np.nan], size=1000)
})

In [158]:
print("Original shape:", df.shape)
### 1. Dropping duplicate rows
df = df.drop_duplicates()
print("After dropping duplicates:", df.shape)


Original shape: (1000, 9)
After dropping duplicates: (1000, 9)


In [159]:
### 2. Missing values
# Fill NaN values in 'Name' with 'Unknown'
df['Name'] = df['Name'].fillna('Unknown')

In [160]:
# Replace negative or NaN ages with median age
df['Age'] = df['Age'].replace(-1, np.nan)
df['Age'] = df['Age'].fillna(df['Age'].median())

In [161]:
# Drop rows with missing emails or empty emails
df['Email'] = df['Email'].replace(['', 'nan'], np.nan)
df = df.dropna(subset=['Email'])

In [162]:
### 3. Convert date column to datetime
df['Signup Date'] = pd.to_datetime(df['Signup Date'], errors='coerce')
df = df.dropna(subset=['Signup Date'])  # Drop rows where date conversion fail

In [163]:
### 4. Clean 'Country' column
df['Country'] = df['Country'].str.upper().str.strip()
df['Country'] = df['Country'].replace(['US'], 'USA')
df['Country'] = df['Country'].replace(['N/A', 'NONE', '', np.nan], 'UNKNOWN')


In [164]:
### 5. Clean 'Subscribed' column to boolean
df['Subscribed'] = df['Subscribed'].str.lower().str.strip()
df['Subscribed'] = df['Subscribed'].map({'yes': True, 'y': True, 'no': False, 'n': False})
df['Subscribed'] = df['Subscribed'].fillna(False) 

C:\Users\Sandriko\AppData\Local\Temp\ipykernel_3492\1922157465.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Subscribed'] = df['Subscribed'].fillna(False)


In [165]:
### 6. Clean 'Notes' column (strip whitespace, fill missing)
df['Notes'] = df['Notes'].astype(str).str.strip()
df['Notes'] = df['Notes'].replace({'nan': 'No Notes', '': 'No Notes'})
df['Notes'] = df['Notes'].fillna('No Notes')

In [166]:
### 7. 
df['Signup_Year'] = df['Signup Date'].dt.year
df['Is_USA'] = df['Country'] == 'USA'

In [167]:
### 8. Rename columns
df = df.rename(columns={
    'User_ID': 'UserID',
    'Signup Date': 'SignupDate'
})

In [168]:
### 9. Reorder columns
columns_order = ['UserID', 'Name', 'Age', 'Email', 'SignupDate', 'Signup_Year',
                 'Country', 'Is_USA', 'Income', 'Subscribed', 'Notes']
df = df[columns_order]

In [169]:
### 10. Round Income 
df['Income'] = df['Income'].round()
df.rename(columns=({'Income': 'Income_USD$'}), inplace=True)

In [170]:
### 11. Sort Values by UserID
df = df.sort_values('UserID').reset_index(drop=True)

In [171]:
### Result
df.head()

,UserID,Name,Age,Email,SignupDate,Signup_Year,Country,Is_USA,Income_USD$,Subscribed,Notes
0,1,David,35.0,test@domain.com,2021-05-20,2021,CANADA,False,34872.0,False,Average
1,4,Unknown,35.0,invalidemail,2021-05-20,2021,CANADA,False,26528.0,False,No Notes
2,6,Bob,40.0,user@example.com,2020-01-01,2020,USA,True,46048.0,True,Good
3,9,Charlie,35.0,test@domain.com,2020-01-01,2020,UNKNOWN,False,45058.0,True,No Notes
4,13,Eve,35.0,user@example.com,2022-03-15,2022,USA,True,57039.0,True,Bad
